# Laboratorio 9
Diego García 22404 <br>
César López 22535 <br>

github: https://github.com/DiegoGarV/Lab9-IA 

## Task 1 - Teoría

1. **Diga cual es la diferencia entre Modelos de Markov y Hidden Markov Models**
Los Modelos de Markov representan un sistema que cambia su estado con el tiempo. Por lo tanto, solo depende de su estado actual y no de como llegó a él. Por su parte, en los HMM los estados no se pueden observar de forma directa. Lo que si se puede observar, son resultados que dan pistas del estado real. Esto hace al HMM más realista en varios problemas. 

2. **Investigue qué son los factorial HMM (Hidden Markov Models)**
Un factorial HMM es una versión más avanzada del HMM. A diferencia del normal, este tiene varias cadenas ocultas de estados que funcionan al mismo tiempo y todas contribuyen en generar el resultado observable. Este es útil para encontrar los estados cuando dependen de multiples causas. Aunque hay varios estados ocultos, el resultado observable sigue siendo solo uno.

3. **Especifique en sus propias palabras el algoritmo Forward Backward para HMM**
Este es un algoritmo que permite calcular que tan probable es que se esté en cada uno de los estados ocultos en todo momento según todas las secuencias de observaciones. Para cada momento y estado se toma un tiempo pasado y se miran las probabilidades de cada estado futuro y se toma un tiempo futuro y se van calculando las probabilidades hacia atras.  

4. **En el algoritmo de Forward Backward, por qué es necesario el paso de Backward (puede escribir ejemplos o casos para responder esta pregunta)**
Porque el paso Forward no logra dar la probabilidad completa de haber estado en alguno de los estados, pues no indica las probabilidades futuras. Por ejemplo: yo quiero saber si la semana pasada mi novia estuvo feliz o triste. Uno de mis observables fue que el miercoles no sonrió. Ahí Forward me diría que lo más probable es que esté triste. Pero si además el jueves y el viernes estuvo sonriendo todo el día, Backward me mostraría que es más probable que esté feliz. Mi conclusión entonces podría ser que aunque el miercoles no sonrió, los demás días si así que lo más seguro es que haya estado feliz. Forward solo limitaría mi visión y me haría tener una conclusión imprecisa.

## Task 2 - Algoritmo Forward Backward en HMM

In [20]:
import random

# Descomentar esta línea para reproducibilidad
# random.seed(42)

class HMM:
    def __init__(self, states, observations, initial_prob, transition_prob, emission_prob):
        self.states = states
        self.observations = observations
        self.initial_prob = initial_prob
        self.transition_prob = transition_prob
        self.emission_prob = emission_prob

    def generate_sequence(self, length):
        sequence = [] # observaciones
        states_sequence = [] # estados ocultos

        current_state = random.choices(self.states, weights=[self.initial_prob[s] for s in self.states])[0]
        states_sequence.append(current_state)

        current_obs = random.choices(self.observations, weights=[self.emission_prob[current_state][o] for o in self.observations])[0]
        sequence.append(current_obs)

        for _ in range(1, length):
            current_state = random.choices(self.states, weights=[self.transition_prob[states_sequence[-1]][s] for s in self.states])[0]
            states_sequence.append(current_state)

            current_obs = random.choices(self.observations, weights=[self.emission_prob[current_state][o] for o in self.observations])[0]
            sequence.append(current_obs)

        return sequence

    def forward(self, obs_seq):
        fwd = []
        f_prev = {}
        # Inicializar con la probabilidad inicial
        for state in self.states:
            f_prev[state] = self.initial_prob[state] * self.emission_prob[state][obs_seq[0]]
        fwd.append(f_prev)

        # Iterar sobre las observaciones siguientes
        for obs in obs_seq[1:]:
            f_curr = {}
            for curr_state in self.states:
                sum_prob = sum(f_prev[prev_state] * self.transition_prob[prev_state][curr_state] for prev_state in self.states)
                f_curr[curr_state] = sum_prob * self.emission_prob[curr_state][obs]
            fwd.append(f_curr)
            f_prev = f_curr

        return fwd

    def backward(self, obs_seq):
        bkw = []
        b_prev = {}
        # Inicializar con 1s
        for state in self.states:
            b_prev[state] = 1.0
        bkw.insert(0, b_prev)

        # Iterar hacia atrás
        for obs in reversed(obs_seq[1:]):
            b_curr = {}
            for curr_state in self.states:
                b_curr[curr_state] = sum(
                    self.transition_prob[curr_state][next_state] *
                    self.emission_prob[next_state][obs] *
                    b_prev[next_state]
                    for next_state in self.states
                )
            bkw.insert(0, b_curr)
            b_prev = b_curr

        return bkw

    def compute_state_probabilities(self, fwd, bkw):
        posterior = []
        for i in range(len(fwd)):
            probs = {}
            norm_factor = sum(fwd[i][state] * bkw[i][state] for state in self.states)
            for state in self.states:
                probs[state] = (fwd[i][state] * bkw[i][state]) / norm_factor
            posterior.append(probs)
        return posterior

In [21]:
# Definición de parametros del modelo HMM
states = ['Rainy', 'Sunny']
observations = ['Sunny', 'Sunny', 'Rainy']
initial_prob = {'Sunny': 0.5, 'Rainy': 0.5}
transation_prob = {'Sunny': {'Sunny': 0.8, 'Rainy': 0.2}, 'Rainy': {'Sunny': 0.4, 'Rainy': 0.6}}
emission_prob = {'Sunny': {'Sunny': 0.8, 'Rainy': 0.2}, 'Rainy': {'Sunny': 0.3, 'Rainy': 0.7}}

In [22]:
# Crear una instancia del modelo HMM
hmm = HMM(states, observations, initial_prob, transation_prob, emission_prob)
print("Modelo HMM creado con éxito.")

Modelo HMM creado con éxito.


In [23]:
# Generar una secuencia de observaciones
obs_sequence = hmm.generate_sequence(5)
print("Secuencia generada:", obs_sequence)

Secuencia generada: ['Sunny', 'Sunny', 'Sunny', 'Sunny', 'Sunny']


In [ ]:
# Calculo de probabilidades Forward
forward_probs = hmm.forward(observations)
for t, probs in enumerate(forward_probs):
    print(f"Tiempo {t}: {probs}")

Tiempo 0: {'Rainy': 0.15, 'Sunny': 0.4}
Tiempo 1: {'Rainy': 0.051000000000000004, 'Sunny': 0.30400000000000005}
Tiempo 2: {'Rainy': 0.06398000000000001, 'Sunny': 0.05272000000000002}


In [ ]:
# Calculo de probabilidades Backward
backward_probs = hmm.backward(observations)
print("\nProbabilidades Backward:")
for t, probs in enumerate(backward_probs):
    print(f"Tiempo {t}: {probs}")


Probabilidades Backward: [{'Rainy': 0.18600000000000003, 'Sunny': 0.22200000000000006}, {'Rainy': 0.5, 'Sunny': 0.30000000000000004}, {'Rainy': 1.0, 'Sunny': 1.0}]


In [29]:
# Cálculo de probabilidades de estado
state_probs = hmm.compute_state_probabilities(forward_probs, backward_probs)
print("\nProbabilidades de estado:", state_probs)


Probabilidades de estado: [{'Rainy': 0.23907455012853465, 'Sunny': 0.7609254498714653}, {'Rainy': 0.21850899742930588, 'Sunny': 0.7814910025706941}, {'Rainy': 0.5482433590402742, 'Sunny': 0.45175664095972584}]
